# Importe de librerias

In [6]:
import lightning as L
from pytorch_lightning.loggers import CSVLogger

from sklearn.model_selection import train_test_split
import torch
import optuna

torch.set_float32_matmul_precision('high')

In [7]:
from dataset import LitPriceData
from trainer import LitTrainer

In [8]:
from utils import splitData

In [9]:
name = "bitcoin"

def objective(trial):   
    hidden_size = trial.suggest_int("hidden_size", 32, 128)
    num_layers = trial.suggest_int("num_layers", 2, 5)
    lr = trial.suggest_float("lr", 1e-4, 1e-2, log=True)
    dropout= trial.suggest_float("dropout", 0.0, 0.5)
    
    sequence_length = trial.suggest_int("sequence_legth", 12, 72, step=12)

    logger = CSVLogger("lightning_logs", name="optuna")
    
    model = LitTrainer(hidden_size=hidden_size, num_layers=num_layers, lr=lr, dropout=dropout)
    X, y = splitData(name, sequence_length=sequence_length)
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
    dataModule = LitPriceData(X_train, y_train, X_val, y_val)
       
    trainer = L.Trainer(max_epochs=75, accelerator="auto", logger=logger, enable_progress_bar=False, enable_model_summary=False, enable_checkpointing=False, log_every_n_steps=2)
    trainer.fit(model=model, datamodule=dataModule)
    
    return trainer.callback_metrics["val_loss"].item()

In [ ]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=200, n_jobs=2)

print("Best trial:")
print(study.best_params)

[I 2025-07-09 03:03:48,836] A new study created in memory with name: no-name-57e891d8-7fdc-4b1c-bb40-cdf9f1ca97c4
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/giorgio6846/miniconda3/envs/TF-ML/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.
/home/giorgio6846/miniconda3/envs/TF-ML/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.

Detected KeyboardInterrupt, attempting graceful shutdown ...
[W 202

NameError: name 'exit' is not defined

In [ ]:
best_params = study.best_params
best_params

{'hidden_size': 76,
 'num_layers': 2,
 'lr': 0.005094100053762949,
 'dropout': 0.4197932446614159,
 'sequence_legth': 12}

In [ ]:
optuna.visualization.plot_optimization_history(study)

In [ ]:
optuna.visualization.plot_parallel_coordinate(study)

In [ ]:
optuna.visualization.plot_param_importances(study)